In [ ]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import pretty_midi
import midi
import matplotlib.patches
import matplotlib
import djitw
import scipy.spatial
import librosa
matplotlib.rc('font',**{'size':16})
%matplotlib inline

In [ ]:
BLUE = '#28ABE3'
GREEN = '#1FDA9A'
ORANGE = '#E8B71A'
GREY = '#CCCCCC'
FIGSIZE = (9, 6)

In [ ]:
plt.figure(figsize=(FIGSIZE[0], FIGSIZE[1]*2))
ax = plt.gca()
t = np.linspace(.1, .9, 9)
plt.vlines(t, 0, 1., linestyles='dashed', alpha=.3, zorder=-1)

vc = .9
words = 'The quick brown fox jumps over the lazy dog'.split(' ')
for x, word in zip(t, words):
    ax.text(x, vc, word, {'family': 'monospace', 'size': 12}, ha='center', va='center')

vc = .7
signal = .08*np.sin(2.4*t*np.pi) + vc
plt.plot(t, signal, 'k.', ms=10)
plt.vlines(t, [s if s > vc else vc for s in signal],
           [s if s < vc else vc for s in signal], lw=1.2) 

vc = .5
a, _ = librosa.load('data/1_A.wav')
N = a.shape[0]
for x in t:
    frame = a[(x - .1)*N:(x + .1)*N]
    spectrum = np.abs(np.fft.rfft(frame))
    spectrum = spectrum[:spectrum.shape[0]/3]
    spectrum = spectrum/3000.
    plt.plot(x + spectrum, np.linspace(vc - .08, vc + .08, spectrum.shape[0]), 'k')

axis = plt.axis()
vc = .3
w = .02
h = .08
dna_names = ['T', 'A', 'C', 'G']
for x in t:
    dna = np.zeros((4, 1))
    n = np.random.randint(0, 4)
    dna[n] = 1
    plt.imshow(dna, interpolation='nearest', extent=(x - w, x + w, vc - h/2, vc + h), cmap=plt.cm.gray)
    plt.plot((x - w, x - w, x + w, x + w, x - w), (vc - h/2, vc + h, vc + h, vc - h/2, vc - h/2), 'k')
    ax.text(x, vc - h/2 - .01, dna_names[n], {'family': 'monospace', 'size': 16}, ha='center', va='top')
plt.axis(axis)

vc = .1
for n, x in enumerate(t):
    im = plt.imread('data/1_video/{}.png'.format(n + 1))
    plt.imshow(im, interpolation='nearest', extent=(x - .03, x + .03, vc - .08, vc + .08))

plt.xlim([0.05, 0.95])
plt.ylim([0, 1])
plt.yticks([])
plt.axis('off')

vc = 0.
words = 'The quick brown fox jumps over the lazy dog'.split(' ')
for n, x in enumerate(t):
    ax.text(x, vc, '$t_{}$'.format(n), {'family': 'monospace', 'size': 16}, ha='center', va='top')

plt.savefig('1-example_sequences.pdf', transparent=True, bbox_inches='tight', pad_inches=0.)

In [ ]:
match_length = 100
crop = match_length/5
def random_walk(N):
    return np.cumsum(np.random.random_integers(-1, 1, N))/np.log(N)
def random_sine(N):
    return np.sin(np.linspace(0, 5*np.pi, N)*np.random.uniform(.9, 1.1) + np.random.uniform(0, 2*np.pi))

match = random_sine(match_length + match_length/10)
query = np.interp(np.arange(match_length),
                  np.arange(match_length + match_length/10),
                  match + .5*random_walk(match_length + match_length/10))
match = match[match_length/10:] + .5*random_walk(match_length)
match = (match - match.mean())/match.std()
query = (query - query.mean())/query.std()

D = np.subtract.outer(match, query[crop/2:-crop/2])**2
p, q, score = djitw.dtw(D, inplace=False)

In [ ]:
ds = 3

plt.figure(figsize=FIGSIZE)
plt.plot(match - match.min(), GREEN, lw=2)
plt.plot(query - query.max(), BLUE, lw=2)

for n in range(0, match_length, ds) + [match_length - 1]:
    plt.plot([n, n], [match[n] - match.min(), query[n] - query.max()], 'k:', lw=2)
    
plt.xlim(-1, plt.axis()[1])
plt.axis('off')
plt.savefig('1-example_distance_unwarped.pdf', transparent=True, bbox_inches='tight', pad_inches=0.1)

plt.figure(figsize=FIGSIZE)
plt.plot(match - match.min(), GREEN, lw=2)
plt.plot(np.arange(crop/2, match_length - crop/2),
         query[crop/2:-crop/2] - query[crop/2:-crop/2].max(), BLUE, lw=2)

for p_n, q_n in zip(p[::ds], q[::ds]):
    
    plt.plot([p_n, q_n + crop/2],
             [match[p_n] - match.min(), query[crop/2:-crop/2][q_n] - query[crop/2:-crop/2].max()],
             'k:', lw=2)

plt.plot([p[-1], q[-1] + crop/2],
         [match[p[-1]] - match.min(), query[crop/2:-crop/2][q[-1]] - query[crop/2:-crop/2].max()],
         'k:', lw=2)

plt.xlim(-1, plt.axis()[1])
plt.axis('off')
plt.savefig('1-example_distance_warped.pdf', transparent=True, bbox_inches='tight', pad_inches=0.1)